In [116]:
import pandas as pd
import ast 
import numpy as np
import csv

In [117]:
data = pd.read_csv('data/extracted_data_final_0_1000.csv')

In [118]:
# Fonction pour réorganiser les profils cards
def reorder_cards(profile_cards):
    try:
        cards_dict = ast.literal_eval(profile_cards)
        
        reordered_cards = {
            'card_1': np.nan,
            'card_2': np.nan,
            'card_3': np.nan,
            'card_4': np.nan,
            'card_5': np.nan
        }

        for key, value in cards_dict.items():
            if value.startswith("Carte et informations d'accès"):
                reordered_cards['card_1'] = value.split('\n') 
            elif value.startswith("Présentation"):
                reordered_cards['card_2'] = value.split('\n') 
            elif value.startswith("Horaires et coordonnées"):
                reordered_cards['card_3'] = value.split('\n') 
            elif value.startswith("Tarifs"):
                reordered_cards['card_4'] = value.split('\n') 
            elif value.startswith("Informations légales"):
                reordered_cards['card_5'] = value.split('\n') 
        
        return reordered_cards
    except:
        return {
            'card_1': np.nan,
            'card_2': np.nan,
            'card_3': np.nan,
            'card_4': np.nan,
            'card_5': np.nan
        }

reordered_cards = data['Profile Cards'].apply(reorder_cards)

reordered_cards_df = pd.json_normalize(reordered_cards)

data_combined = pd.concat([data, reordered_cards_df], axis=1)

data_combined.drop(columns = 'Profile Cards', inplace=True)

In [119]:
#Pour rendre la localisation plus claire
def extract_lat_lng(map_location):
    try:
        location_dict = ast.literal_eval(map_location)
        return (location_dict['lat'], location_dict['lng'])
    except:
        return (None, None)

data_combined['Latitude'], data_combined['Longitude'] = zip(*data_combined['Map Location'].apply(extract_lat_lng))

data_combined.drop(columns = 'Map Location', inplace=True)


In [120]:
# Fonction pour traiter la colonne 'Skills'
def process_skills(skills):
    if skills == 'Skills missing':
        return np.nan
    else:
        return skills.split('\n')

data_combined['Skills Processed'] = data_combined['Skills'].apply(process_skills)

data_combined.drop(columns = 'Skills', inplace=True)

In [121]:
data_combined['Specialty'].value_counts()

Specialty
Chirurgien-dentiste                                                 152
Médecin généraliste                                                 104
Ophtalmologue                                                        75
Cabinet médical                                                      56
Spécialiste en chirurgie plastique reconstructrice et esthétique     40
                                                                   ... 
Chirurgien oral                                                       1
Acupuncteur                                                           1
Addictologue                                                          1
Chirurgien cancérologue                                               1
Diététicien                                                           1
Name: count, Length: 78, dtype: int64

In [122]:
data_combined['RPPS'] = data_combined['card_5'].apply(lambda x: x[2] if isinstance(x, list) and len(x) > 2 else np.nan)

data_combined.drop(columns = 'card_5', inplace = True)


In [123]:
data_combined['Nb Skills'] = data_combined['Skills Processed'].apply(lambda x: len(x)-1 if isinstance(x, list) else np.nan)

In [124]:
data_combined['Présentation']=''
data_combined['Langues parlées']=''
data_combined['Diplômes nationaux et universitaires']=''
data_combined['Autres formations']=''
data_combined['Expériences']=''
data_combined['Site web']=''
data_combined['Travaux et publications']=''
data_combined['Prix et distinctions']=''
data_combined['site']=0

liste_mots_clefs = ['Présentation','Langues parlées','Diplômes nationaux et universitaires','Formations','Autres formations','Expériences', 'Site web', 'Travaux et publications', 'Prix et distinctions']

for i in range(data_combined.shape[0]):
    
    if isinstance(data_combined['card_2'].iloc[i],list) : 
        liste_card2 = data_combined['card_2'].iloc[i][1:]
        sous_liste = ''
        label = liste_mots_clefs[0]
        for x in liste_card2 :
            if x in liste_mots_clefs :
                data_combined.loc[i, label] = sous_liste
                label = x
                sous_liste = ''
            elif x=='Voir le site':
                data_combined.loc[i, 'site'] = 1
            elif x=='▾ Voir plus':
                None
            else :
                sous_liste = sous_liste+(x)

In [125]:
data_combined['Diplômes nationaux et universitaires b']=data_combined['Diplômes nationaux et universitaires'].apply(lambda x: 1 if len(x)>0 else 0)

In [126]:
data_combined['Nb caractères présentation'] = data_combined['Présentation'].apply(lambda x: len(x))

In [127]:
data_combined['Autres formations b']=data_combined['Autres formations'].apply(lambda x: 1 if len(x)>0 else 0)

In [128]:
data_combined['Nb langues']=0

for i in range(data_combined.shape[0]):
    if isinstance(data_combined['Langues parlées'].iloc[i],str) : 
        liste_langue = (data_combined['Langues parlées'].iloc[i]).split()
        count = 0
        for x in liste_langue :
            
            if  x != 'et'  :
                count +=1
        data_combined.loc[i,'Nb langues']=count

In [129]:
data_combined['Expériences b']=data_combined['Expériences'].apply(lambda x: 1 if len(x)>0 else 0)

In [130]:
liste1 = ["Carte et informations d'accès",'Moyens de transport','Parking public','Informations pratiques',"J'autorise le traitement d'informations (dont mon adresse IP) et leur transfert hors UE par Google Maps (USA) afin d’afficher la carte.En savoir plus", 'sur la collecte et le traitement des données par Google','AFFICHER LA CARTE']
def carac1(card1):
    nb=0
    for phrase in card1:
        if  phrase not in liste1:
            nb += len(phrase)
    return nb
data_combined['Carac_card_1'] = data_combined['card_1'].apply(lambda x: carac1(x)if isinstance(x, list) else np.nan)

In [131]:
def transport(card1):
    if isinstance(card1, list):
        if 'Moyens de transport' in card1:
            return 1
        else:
            return 0
    else:
        return 0
data_combined['Transport'] = data_combined['card_1'].apply(transport)

In [132]:
def parking(card1):
    if isinstance(card1, list):
        if 'Parking public' in card1:
            return 1
        else:
            return 0
    else:
        return 0
data_combined['Parking'] = data_combined['card_1'].apply(parking)

In [133]:
def infos(card1):
    if isinstance(card1, list):
        if 'Informations pratiques' in card1:
            return 1
        else:
            return 0
    else:
        return 0
data_combined['Infos'] = data_combined['card_1'].apply(infos)

In [134]:
def coordonnées(card3):
    if isinstance(card3, list):
        if 'Coordonnées' in card3:
            return 1
        else:
            return 0
    else:
        return 0
data_combined['Coordonnées'] = data_combined['card_3'].apply(coordonnées) 

In [135]:
def sansRDV(card3):
    if isinstance(card3, list):
        if 'Consultations sans rendez-vous' in card3:
            return 1
        else:
            return 0
    else:
        return 0
data_combined['Sans RDV'] = data_combined['card_3'].apply(sansRDV)

In [136]:
data_combined.head()

,Doctor Name,Specialty,URL,card_1,card_2,card_3,card_4,Latitude,Longitude,Skills Processed,...,Nb caractères présentation,Autres formations b,Nb langues,Expériences b,Carac_card_1,Transport,Parking,Infos,Coordonnées,Sans RDV
0,Dr Marian AGACHI,Neurochirurgien,https://www.doctolib.fr/neurochirurgien/paris/...,"[Carte et informations d'accès, Centre médico-...","[Présentation, Le docteur Marian Agachi vous a...","[Horaires et coordonnées, Horaires d'ouverture...",NaN,48.878328,2.431034,"[Expertises et actes, Neurochirurgie du rachis]",...,748,0,3,0,154.0,1,0,1,1,0
1,Dr Anne Vaillant Moga,Médecin généraliste,https://www.doctolib.fr/medecin-generaliste/pa...,"[Carte et informations d'accès, Dr Anne Moga (...","[Présentation, Située au cœur du seizieme arr...","[Horaires et coordonnées, Horaires d'ouverture...","[Tarifs, Consultation préalable de médecine es...",48.869218,2.285468,"[Expertises et actes, Acné, Allergie, Apnée du...",...,2538,1,2,1,180.0,1,0,1,0,0
2,Dr Gabriel Ohana,Chirurgien-dentiste,https://www.doctolib.fr/dentiste/courbevoie/ga...,"[Carte et informations d'accès, Cabinet dentai...","[Présentation, Le docteur Gabriel Ohana vous a...","[Horaires et coordonnées, Horaires d'ouverture...","[Tarifs, Consultation dentaire, 23 €, Traiteme...",48.894148,2.250698,"[Expertises et actes, Chirurgie buccale, Proth...",...,481,1,1,0,240.0,1,0,1,1,0
3,Dr Thomas BAMBERGER,Dermatologue et vénérologue,https://www.doctolib.fr/dermatologue/gentilly/...,"[Carte et informations d'accès, Villa Amélie, ...","[Présentation, Le docteur Thomas Bamberger vou...","[Horaires et coordonnées, Horaires d'ouverture...","[Tarifs, Consultation, 31,50 € à 56,50 €, Cont...",48.815433,2.343017,NaN,...,407,0,1,1,155.0,1,0,1,1,0
4,Dr Layla Chatila,Chirurgien-dentiste,https://www.doctolib.fr/dentiste/paris/layla-c...,"[Carte et informations d'accès, 5 rue François...","[Présentation, Le docteur Layla Chatila vous a...","[Horaires et coordonnées, Coordonnées, 01 45 2...","[Tarifs, Consultation dentaire, 23 €, Ces hono...",48.858804,2.274392,NaN,...,246,0,1,0,202.0,1,1,1,1,0


Ouverture annuaire santé

In [137]:
annuaire = pd.read_csv('data/Annuaire_Sante.csv')
annuaire.head()

/var/folders/rf/dqdwsk6s06ngm9b4j17fkb2r0000gn/T/ipykernel_70643/3626557920.py:1: DtypeWarning: Columns (24,25,26,27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  annuaire = pd.read_csv('data/Annuaire_Sante.csv')


,typedidentifiantpp,identifiantpp,identificationnationalepp,codecivilité,codeprofession,libelléprofession,codemodeexercice,libellémodeexercice,numérosiretsite,numérosirensite,...,libellérôle\n,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28
0,0,10001410,10001410,M,71,Ostéopathe,L,"""Lib",indép,artis,...,NaN,NaN,NaN,\n,NaN,NaN,NaN,NaN,NaN,NaN
1,0,10003887,10003887,MME,71,Ostéopathe,S,Salarié,26010004500012,NaN,...,\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,10005478,10005478,MME,71,Ostéopathe,L,"""Lib",indép,artis,...,NaN,NaN,NaN,\n,NaN,NaN,NaN,NaN,NaN,NaN
3,0,10005684,10005684,MME,71,Ostéopathe,L,"""Lib",indép,artis,...,NaN,NaN,NaN,\n,NaN,NaN,NaN,NaN,NaN,NaN
4,0,19111228,19111228,MME,91,Orthophoniste,L,"""Lib",indép,artis,...,SA07,Cabinet individuel,FON-01,Titulaire de cabinet\n,NaN,NaN,NaN,NaN,NaN,NaN
